In [ ]:
# Based on this paper - https://arxiv.org/pdf/1706.03762.pdf
# Might want to move layer norm inside the residual block - https://arxiv.org/pdf/2002.04745.pdf
# Layer normalization - https://arxiv.org/pdf/1607.06450.pdf
# TODO: Investigate learning rate warmup - https://arxiv.org/abs/2002.04745
#!pip install torch datasets wandb

In [ ]:
import numpy as np
import torch
from torch import nn
import sys
import os
import math
import einops
import torch.nn.functional as F

sys.path.append(os.path.abspath("../../data"))
sys.path.append(os.path.abspath("../../nnets"))
from net_utils import get_module_list

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
SP_VOCAB_SIZE = 5000

In [ ]:
# TODO: Fix dataset to work with new format
# TODO: Improve dataset to optimize padding and other things (drop incomplete, etc)
from text_data import WikiTextDataset
from torch.utils.data import DataLoader

train_wrapper = WikiTextDataset(tokenizer_vocab=SP_VOCAB_SIZE, download_split_pct="2%", model_max_length=128)
train_dataset = train_wrapper.process_dataset()
train_dataset = train_dataset.train_test_split(test_size=0.005)

train_split = train_dataset["train"]
train = DataLoader(train_split, batch_size=BATCH_SIZE)

valid_split = train_dataset["test"]
valid = DataLoader(valid_split, batch_size=BATCH_SIZE)

In [ ]:
# Add in ROPE embedding
class ROPE(nn.Module):
    def __init__(self, embedding_dim, seq_len):
        super(ROPE, self).__init__()
        self.embedding_dim = embedding_dim
        self.seq_len = seq_len

        self.cos_embeds = torch.zeros(seq_len, embedding_dim, device=DEVICE)
        self.sin_embeds = torch.zeros(seq_len, embedding_dim, device=DEVICE)

        embed_pos = 10000 ** (-2 * torch.ceil((torch.arange(0, embedding_dim) + 1) / 2) / embedding_dim)
        for i in range(0, seq_len):
            self.cos_embeds[i] = torch.cos((i + 1) * embed_pos)
            self.sin_embeds[i] = torch.sin((i + 1) * embed_pos)

        self.indices = torch.zeros(self.embedding_dim, device=DEVICE, dtype=torch.long)
        self.mask = torch.zeros(self.embedding_dim, device=DEVICE, dtype=torch.int)
        for i in range(0, embedding_dim, 2):
            self.indices[i] = i + 1
            self.indices[i+1] = i

            self.mask[i] = -1
            self.mask[i+1] = 1


    def rotate(self, x):
        return x[...,self.indices] * self.mask

    def forward(self, x):
        current_val = x * self.cos_embeds[:x.shape[-2],:]
        next_val = self.rotate(x) * self.sin_embeds[:x.shape[-2],:]
        return current_val + next_val


class MultiHeadAttention(nn.Module):
    def __init__(self, input_units, attention_heads, mask=False):
        super(MultiHeadAttention, self).__init__()
        self.input_units = input_units
        self.attention_heads = attention_heads
        self.head_units = int(input_units/attention_heads)
        self.mask = mask

        k = math.sqrt(1/self.input_units)
        # Drop bias
        # Single kv head
        self.kv_proj_weight = nn.Parameter(torch.rand(2, input_units, self.head_units) * 2 * k - k)
        self.q_proj_weight = nn.Parameter(torch.rand(input_units, self.attention_heads * self.head_units) * 2 * k - k)
        self.out_proj_weight = nn.Parameter(torch.rand(self.attention_heads * self.head_units, input_units) * 2 * k - k)

        # 1024 is max sequence length
        self.rope = ROPE(self.head_units, 1024)

    def forward(self, queries, keys, values):
        # convert to 4d tensor with batch_size, attn_heads, seq_len, embedding_dim
        proj_queries = torch.einsum("...se, eo->...so", queries, self.q_proj_weight)
        proj_queries = proj_queries.view(queries.shape[0], queries.shape[1], self.attention_heads, self.head_units).swapaxes(1,2)
        proj_queries = self.rope(proj_queries)

        proj_keys = torch.einsum("...se, eo->...so", keys, self.kv_proj_weight[0])
        proj_keys = proj_keys.view(keys.shape[0], keys.shape[1], self.head_units)
        proj_keys = self.rope(proj_keys)

        proj_values = torch.einsum("...se, eo->...so", values, self.kv_proj_weight[0])
        proj_values = proj_values.view(values.shape[0], values.shape[1], self.head_units)

        attention = torch.einsum("baqh, bhk->baqk", proj_queries, torch.transpose(proj_keys, -1, -2)) / np.sqrt(proj_keys.shape[-1])
        if self.mask:
            # Prevent decoder queries from looking at tokens that come after
            # Do this by setting attention to negative infinity, so it is softmaxed to zero in the next step
            mask = torch.full((attention.shape[-2], attention.shape[-1]), -torch.inf, device=DEVICE)
            attention += torch.triu(mask, diagonal=1)

        # Softmax on last dimension
        # Sequence-wise softmax, so attention between one sequence and other sequences sums to 1
        attention = torch.softmax(attention, dim=-1)
        weighted_values = torch.einsum("baqk, bke->baqe", attention, proj_values)

        # Swap attention head and sequence axis, then reshape to batch, seq, embedding
        weighted_values = weighted_values.swapaxes(1,2).reshape(queries.shape[0], queries.shape[1], -1)
        weighted_values = torch.einsum("...se, eo->...so", weighted_values, self.out_proj_weight)
        return weighted_values

In [ ]:
class SwiGLU(nn.Module):
    def __init__(self, input_units, hidden_units):
        super(SwiGLU, self).__init__()
        self.linear1 = nn.Linear(input_units, hidden_units, bias=False)
        self.linear2 = nn.Linear(input_units, hidden_units, bias=False)
        self.linear3 = nn.Linear(hidden_units, input_units, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x1 = self.linear1(x)
        swish = self.sigmoid(x1) * x1
        x2 = self.linear2(x)
        swiglu = self.linear3(swish * x2)
        return swiglu

class DecoderBlock(nn.Module):
    def __init__(self, input_units, attention_heads, hidden_units=2048, dropout_p=.1):
        super(DecoderBlock, self).__init__()
        self.in_attn = MultiHeadAttention(input_units, attention_heads, mask=True)
        self.dropouts = get_module_list(2, nn.Dropout, dropout_p)
        # Drop bias
        self.lns = get_module_list(2, nn.LayerNorm, input_units)
        # Switch to swiglu from two linear layers
        self.swiglu = SwiGLU(input_units, hidden_units)

    def forward(self, x):
        weighted_values = self.dropouts[0](self.in_attn(x, x, x))
        # Pre normalization
        x = x + self.lns[0](weighted_values)

        reprojected = self.dropouts[1](self.swiglu(x))
        # Pre normalization
        x = x + self.lns[1](reprojected)
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, input_units, hidden_units, attention_heads, max_len=256, blocks=1):
        super(Transformer, self).__init__()
        self.blocks = blocks
        self.dropouts = get_module_list(2, nn.Dropout, .1)
        self.decoders = get_module_list(blocks, DecoderBlock, hidden_units, attention_heads)

        self.embedding = nn.Parameter(torch.empty(input_units, hidden_units))
        nn.init.xavier_uniform_(self.embedding)
        self.input_units = input_units

    # Tie input output weights
    def embed(self, x, reverse=False):
        if reverse:
            return x @ self.embedding.T
        else:
            embedded = self.embedding[x.to(torch.long).view(-1)]
            return embedded.view(x.shape[0], x.shape[1], -1)


    def forward(self, x):
        dec_outputs = self.dropouts[1](self.embed(x))
        for i in range(self.blocks):
            dec_outputs = self.decoders[i](dec_outputs)

        token_vectors = self.embed(dec_outputs, reverse=True)
        return token_vectors

In [ ]:
def replace_newline(text):
    return text.replace('\n', ' ').replace('\r', '')

def generate(sequence, pred, wrapper):
    prompts = wrapper.decode_ids(sequence.cpu())
    texts = wrapper.decode_ids(torch.argmax(pred, dim=2).cpu())

    displays = []
    for p, t in zip(prompts, texts):
        displays.append(f"{replace_newline(p)} \x1b[31m{replace_newline(t)}\x1b[0m")
    return displays

In [ ]:
from tqdm.auto import tqdm
import wandb
import time

wandb.init(project="decoder-only", notes="Change dataset", name="change-dataset")

# TODO: Profile and improve perf - https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html
model = Transformer(SP_VOCAB_SIZE, 512, 8, blocks=6).to(DEVICE)
loss_fn = nn.CrossEntropyLoss(ignore_index=train_wrapper.tokenizer.pad_token_id)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
wandb.watch(model, log_freq=100)

In [ ]:
EPOCHS = 100
DISPLAY_BATCHES = 2
PRINT_VALID = True
ACCUMULATE_STEPS = 1

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    match_pct = 0
    optimizer.zero_grad(set_to_none=True)
    start = time.time()
    for idx, batch in enumerate(tqdm(train)):
        sequence = batch["input_ids"].to(torch.long)
        target = torch.roll(sequence, -1, -1)
        # TODO: Look into replace this with eos id instead
        target[:,-1] = train_wrapper.tokenizer.pad_token_id
        pred = model(sequence.to(DEVICE))

        loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.reshape(-1).to(DEVICE))
        loss.backward()
        train_loss += loss.item()

        # Accumulate gradients
        # This seems to perform worse than no accumulation over a
        # small data set.  Test with larger set.
        if idx % ACCUMULATE_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
    end = time.time()

    with torch.no_grad():
        mean_loss = train_loss / len(train) / BATCH_SIZE
        wandb.log({"loss": mean_loss, "epoch_time": end - start})
        print(f"Epoch {epoch} train loss: {mean_loss}")
        sents = generate(sequence, pred, train_wrapper)
        for sent in sents[:DISPLAY_BATCHES]:
            print(sent)

        if PRINT_VALID and epoch % 1 ==0:
            # Compute validation loss.  Unless you have a lot of training data, the validation loss won't decrease.
            valid_loss = 0
            # Deactivate dropout layers
            model.eval()
            for batch in tqdm(valid):
                # Inference token by tokens
                sequence = batch["input_ids"].to(torch.long)
                target = torch.roll(sequence, -1, -1)
                target[:,-1] = train_wrapper.tokenizer.pad_token_id

                outputs = sequence[:,:50].to(DEVICE)
                # TODO: Investigate memory leak with valid generation
                for i in range(sequence.shape[1]):
                    pred = model(outputs)
                    last_output = torch.argmax(pred, dim=2)
                    outputs = torch.cat((outputs, last_output[:,-1:]), dim=1)

                loss = loss_fn(pred.reshape(-1, pred.shape[-1]), target.reshape(-1).to(DEVICE))
                valid_loss += loss.item()
            mean_loss = valid_loss / len(valid) / BATCH_SIZE
            wandb.log({"valid_loss": mean_loss})
            print(f"Valid loss: {mean_loss}")
            sents = generate(sequence, pred, train_wrapper)
            for sent in sents[:DISPLAY_BATCHES]:
                print(sent)
            # Reactivate dropout
            model.train()

In [ ]:

from torchinfo import summary

print(summary(model))

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU], record_shapes=True, ) as prof:
    model(sequence.to(DEVICE), prev_target.to(DEVICE))

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

In [ ]:
from einops import rearrange

x = torch.rand(4, 3)
x[torch.tensor([0,1,0])]